# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd


In [3]:
# Your code here; create the connection and cursor
conn=sqlite3.connect('data.sqlite')
cur=conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [4]:
# Your code here; use a subquery. No join is necessary 
cur.execute("""SELECT customerNumber, contactLastName,contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT customerNumber
                                     FROM orders
                                     WHERE orderDate = '2003-01-31');""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [7]:
cur.execute("""SELECT * FROM orders""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141
...,...,...,...,...,...,...,...
321,10421,2005-05-29,2005-06-06,,In Process,Custom shipping instructions were sent to ware...,124
322,10422,2005-05-30,2005-06-11,,In Process,,157
323,10423,2005-05-30,2005-06-05,,In Process,,314
324,10424,2005-05-31,2005-06-08,,In Process,,141


In [8]:
cur.execute("""SELECT * FROM products""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00
...,...,...,...,...,...,...,...,...,...
105,S700_3505,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17
106,S700_3962,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31
107,S700_4002,American Airlines: MD-11S,Planes,1:700,Second Gear Diecast,Polished finish. Exact replia with official lo...,8820,36.27,74.03
108,S72_1253,Boeing X-32A JSF,Planes,1:72,Motor City Art Classics,"10"" Wingspan with retractable landing gears.Co...",4857,32.77,49.66


In [9]:
cur.execute("""SELECT * FROM orderDetails""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4
...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9
2992,10425,S24_2840,31,31.82,5
2993,10425,S32_1268,41,83.79,11
2994,10425,S32_2509,11,50.32,6


In [20]:
cur.execute("""SELECT productName, quantityOrdered FROM products
            JOIN orderDetails
            USING (productCode)
            JOIN orders
            USING (orderNumber)
            GROUP BY productName
            ORDER BY productName DESC
            ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productName,quantityOrdered
0,The USS Constitution Ship,31
1,The Titanic,39
2,The Schooner Bluenose,41
3,The Queen Mary,22
4,The Mayflower,29
...,...,...
104,1903 Ford Model A,22
105,1900s Vintage Tri-Plane,26
106,1900s Vintage Bi-Plane,49
107,18th century schooner,43


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [25]:
# Your code here
cur.execute("""SELECT productName, COUNT(orderNumber) as numberOrders, SUM(quantityOrdered) AS totalSold
            FROM products
            JOIN orderDetails
            USING (productCode)
            GROUP BY productName
            ORDER BY totalSold DESC
            LIMIT 15;
            """)
            
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productName,numberOrders,totalSold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
5,1940s Ford truck,28,1061
6,1969 Harley Davidson Ultimate Chopper,28,1057
7,1957 Chevy Pickup,28,1056
8,1964 Mercedes Tour Bus,28,1053
9,1956 Porsche 356A Coupe,27,1052


In [26]:
cur.execute("""SELECT * FROM customers""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,,Philadelphia,PA,71270,USA,1323,72600
118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,,Brisbane,CA,94217,USA,1165,60300
119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,,London,,WA1 1DP,UK,1501,43300
120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [30]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute("""SELECT productName, COUNT (DISTINCT customerNumber) AS numCustomersOrdered
            FROM products
            JOIN orderDetails
            USING (productCode)
            JOIN orders
            USING (orderNumber)
            GROUP BY productName
            ORDER BY numCustomersOrdered DESC
            LIMIT 15""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df


,productName,numCustomersOrdered
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
5,HMS Bounty,26
6,Corsair F4U ( Bird Cage),26
7,American Airlines: MD-11S,26
8,1999 Yamaha Speed Boat,26
9,1996 Moto Guzzi 1100i,26


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [46]:
# Your code here
cur.execute("""SELECT DISTINCT employeeNumber, lastName, firstName, o.city, officeCode
            FROM employees e
            JOIN offices o
            USING (officeCode)
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders
            USING (customerNumber)
            JOIN orderDetails
            USING (orderNumber)
            WHERE productCode IN (SELECT productCode
                                  FROM products
                                  JOIN orderDetails
                                  USING (productCode)
                                  JOIN orders
                                  USING (orderNumber)
                                  GROUP BY productCode
                                  HAVING COUNT (DISTINCT customerNumber) < 20)
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,city,officeCode
0,1370,Hernandez,Gerard,Paris,4
1,1501,Bott,Larry,London,7
2,1337,Bondur,Loui,Paris,4
3,1166,Thompson,Leslie,San Francisco,1
4,1286,Tseng,Foon Yue,NYC,3
5,1612,Marsh,Peter,Sydney,6
6,1611,Fixter,Andy,Sydney,6
7,1401,Castillo,Pamela,Paris,4
8,1621,Nishi,Mami,Tokyo,5
9,1323,Vanauf,George,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [48]:
# Your code here
cur.execute("""SELECT employeeNumber, firstName, lastName, 
            count(DISTINCT customerNumber) AS num_customers
            FROM employees e
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            GROUP BY employeeNumber
            ORDER BY num_customers DESC
            HAVING creditLimit > 15000""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

OperationalError: near "HAVING": syntax error

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!